# Reproducible research project
Authors:
Zuzanna Miazio
Filip Szymański
Kacper Sokołowski

In [4]:
import numpy as np 
import pandas as pd 
pd.set_option('display.max_columns', 100)
from pathlib import Path

In [5]:
data_path = Path.cwd() / "data/bank_data_prediction_task.csv"
df = pd.read_csv(data_path)

C:\Users\HP\AppData\Local\Temp\ipykernel_12204\1064735160.py:2: DtypeWarning: Columns (8,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_path)


In [6]:
df_raw = df.copy()

### Initial data inspection

In [7]:
df.sample(10).head(10)

,Unnamed: 0,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y,test_control_flag
34047,34048,43,blue-collar,married,basic.6y,no,yes,no,NaN,NaN,NaN,NaN,NaN,999,0,nonexistent,-1.8,93.075,-47.1,1.405,5099.1,no,control group
9317,9318,57,retired,married,professional.course,no,no,no,cellular,aug,wed,508.0,4.0,999,0,nonexistent,1.4,93.444,-36.1,4.964,5228.1,yes,campaign group
7831,7832,42,management,married,university.degree,unknown,yes,no,cellular,aug,thu,109.0,5.0,999,0,nonexistent,1.4,93.444,-36.1,4.968,5228.1,no,campaign group
1952,1953,30,technician,married,professional.course,no,yes,no,telephone,may,wed,265.0,1.0,999,0,nonexistent,1.1,93.994,-36.4,4.858,5191.0,no,campaign group
18534,18535,56,blue-collar,married,basic.9y,unknown,yes,no,NaN,NaN,NaN,NaN,NaN,999,0,nonexistent,1.1,93.994,-36.4,4.860,5191.0,no,control group
10354,10355,31,self-employed,married,university.degree,no,yes,no,cellular,nov,wed,118.0,1.0,999,0,nonexistent,-0.1,93.200,-42.0,4.120,5195.8,no,campaign group
21031,21032,46,blue-collar,married,basic.9y,no,yes,no,NaN,NaN,NaN,NaN,NaN,999,0,nonexistent,1.1,93.994,-36.4,4.864,5191.0,no,control group
16724,16725,35,admin.,married,university.degree,no,no,no,NaN,NaN,NaN,NaN,NaN,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no,control group
24387,24388,25,services,married,high.school,no,no,no,NaN,NaN,NaN,NaN,NaN,999,0,nonexistent,1.4,93.918,-42.7,4.962,5228.1,no,control group
14000,14001,29,blue-collar,divorced,high.school,no,no,no,cellular,may,fri,760.0,1.0,999,0,nonexistent,-1.8,92.893,-46.2,1.250,5099.1,yes,campaign group


### renaming dataset columns 

In [8]:
df = df.rename(columns={df.columns[0]: 'client_id'})

In [9]:
colnames_to_change = ["emp.var.rate", "cons.price.idx", "cons.conf.idx", "nr.employed"]
dict = {}

for col in colnames_to_change:
    new = col.replace(".", "_")
    dict[col] = new
    
df = df.rename(columns = dict)

### Data description


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 23 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   client_id          41188 non-null  int64  
 1   age                41188 non-null  int64  
 2   job                41188 non-null  object 
 3   marital            41188 non-null  object 
 4   education          41188 non-null  object 
 5   default            41188 non-null  object 
 6   housing            41188 non-null  object 
 7   loan               41188 non-null  object 
 8   contact            16476 non-null  object 
 9   month              16476 non-null  object 
 10  day_of_week        16476 non-null  object 
 11  duration           16476 non-null  float64
 12  campaign           16476 non-null  float64
 13  pdays              41188 non-null  int64  
 14  previous           41188 non-null  int64  
 15  poutcome           41188 non-null  object 
 16  emp_var_rate       411

In [ ]:
df['test_control_flag'].value_counts()

control group     24712
campaign group    16476
Name: test_control_flag, dtype: int64

In [ ]:
df['cons_price_idx'].value_counts()

93.99    7719
93.92    6643
92.89    5762
93.44    5147
94.47    4353
93.20    3588
93.08    2440
92.20     764
92.96     709
92.43     444
92.65     356
94.22     311
94.20     301
92.84     281
92.38     265
93.37     260
94.03     231
94.06     227
93.88     211
94.60     204
92.47     176
92.71     172
93.75     172
94.77     127
93.80      66
92.76       9
Name: cons_price_idx, dtype: int64

In [ ]:
null_cons_price_idx = df[df['cons_price_idx'].isnull()]
null_cons_price_idx['test_control_flag'].value_counts()

control group    250
Name: test_control_flag, dtype: int64

In [ ]:
df['cons_price_idx'].fillna(df['cons_price_idx'].median(), inplace=True)

In [ ]:
df.describe()

,client_id,age,duration,campaign,pdays,previous,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed
count,41188.00,41188.00,16476.00,16476.00,41188.00,41188.00,41188.00,41188.00,41188.00,41188.00,41188.00
mean,20594.50,40.02,259.66,2.55,962.48,0.17,0.08,93.58,-40.50,3.62,5167.04
std,11890.10,10.42,260.66,2.70,186.91,0.49,1.57,0.58,4.63,1.73,72.25
min,1.00,17.00,0.00,1.00,0.00,0.00,-3.40,92.20,-50.80,0.63,4963.60
25%,10297.75,32.00,103.00,1.00,999.00,0.00,-1.80,93.08,-42.70,1.34,5099.10
50%,20594.50,38.00,180.00,2.00,999.00,0.00,1.10,93.75,-41.80,4.86,5191.00
75%,30891.25,47.00,321.00,3.00,999.00,0.00,1.40,93.99,-36.40,4.96,5228.10
max,41188.00,98.00,4918.00,43.00,999.00,7.00,1.40,94.77,-26.90,5.04,5228.10


In [ ]:
df.describe(include = 'object')

,job,marital,education,default,housing,loan,contact,month,day_of_week,poutcome,y,test_control_flag
count,41188,41188,41188,41188,41188,41188,16476,16476,16476,41188,41188,41188
unique,12,4,8,3,3,3,2,10,5,3,2,2
top,admin.,married,university.degree,no,yes,no,cellular,may,thu,nonexistent,no,control group
freq,10422,24928,12168,32588,21576,33950,10481,5497,3522,35563,36020,24712


In [ ]:
df['test_control_flag'].value_counts()

control group     24712
campaign group    16476
Name: test_control_flag, dtype: int64

In [ ]:
df['pdays'].replace(999, np.nan, inplace = True)

In [ ]:
temp_df = df.drop(['client_id'], axis=1)
temp_df.duplicated().sum()


2306

In [ ]:
duplicates = temp_df[temp_df.duplicated(keep=False)]
display(duplicates)


,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,y,test_control_flag
496,39,blue-collar,married,basic.6y,no,no,no,telephone,may,thu,124.00,1.00,NaN,0,nonexistent,1.10,93.99,-36.40,4.86,5191.00,no,campaign group
497,39,blue-collar,married,basic.6y,no,no,no,telephone,may,thu,124.00,1.00,NaN,0,nonexistent,1.10,93.99,-36.40,4.86,5191.00,no,campaign group
10057,39,admin.,married,university.degree,no,no,no,cellular,nov,tue,123.00,2.00,NaN,0,nonexistent,-0.10,93.20,-42.00,4.15,5195.80,no,campaign group
10069,39,admin.,married,university.degree,no,no,no,cellular,nov,tue,123.00,2.00,NaN,0,nonexistent,-0.10,93.20,-42.00,4.15,5195.80,no,campaign group
16480,59,admin.,married,professional.course,no,no,no,NaN,NaN,NaN,NaN,NaN,NaN,0,nonexistent,1.10,93.99,-36.40,4.86,5191.00,no,control group
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40713,41,entrepreneur,married,university.degree,no,yes,no,NaN,NaN,NaN,NaN,NaN,NaN,0,nonexistent,-1.70,94.03,-38.30,0.90,4991.60,yes,control group
40912,33,technician,married,university.degree,no,yes,no,NaN,NaN,NaN,NaN,NaN,NaN,1,failure,-1.10,94.20,-37.50,0.88,4963.60,no,control group
40916,33,technician,married,university.degree,no,yes,no,NaN,NaN,NaN,NaN,NaN,NaN,1,failure,-1.10,94.20,-37.50,0.88,4963.60,no,control group
40954,35,technician,married,professional.course,no,yes,no,NaN,NaN,NaN,NaN,NaN,NaN,2,failure,-1.10,94.20,-37.50,0.88,4963.60,no,control group


In [ ]:
duplicates['test_control_flag'].value_counts()

control group     4142
campaign group       4
Name: test_control_flag, dtype: int64

In [ ]:
duplicates[duplicates['test_control_flag'] == 'campaign group']

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,y,test_control_flag
496,39,blue-collar,married,basic.6y,no,no,no,telephone,may,thu,124.00,1.00,NaN,0,nonexistent,1.10,93.99,-36.40,4.86,5191.00,no,campaign group
497,39,blue-collar,married,basic.6y,no,no,no,telephone,may,thu,124.00,1.00,NaN,0,nonexistent,1.10,93.99,-36.40,4.86,5191.00,no,campaign group
10057,39,admin.,married,university.degree,no,no,no,cellular,nov,tue,123.00,2.00,NaN,0,nonexistent,-0.10,93.20,-42.00,4.15,5195.80,no,campaign group
10069,39,admin.,married,university.degree,no,no,no,cellular,nov,tue,123.00,2.00,NaN,0,nonexistent,-0.10,93.20,-42.00,4.15,5195.80,no,campaign group


In [ ]:
columns_to_check = df.columns[df.columns != 'client_id']
df.drop_duplicates(subset = columns_to_check, keep='first', inplace=True)

In [ ]:
df.duplicated().sum()

0